<a href="https://colab.research.google.com/github/jdmedinatobon/proyectoMachineLearning/blob/master/CrearArchivoDescriptores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile

datosZip = zipfile.ZipFile("/content/drive/My Drive/Datos.zip", 'r')
datosZip.extractall()
datosZip.close()

In [5]:
#En esta celda se incluyen los comandos necesarios para instalar las librerias requeridas.
#Esta solo se debe ejecutar cada vez que se inicia el Runtime.
!pip install mahotas
!pip install progressbar2

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Created wheel for mahotas: filename=mahotas-1.4.9-cp36-cp36m-linux_x86_64.whl size=4264075 sha256=2cd412d7445b602d562e90c6040e8e72b6e7fa34ade090e7a61ac774fc359599
  Stored in directory: /root/.cache/pip/wheels/02/e7/50/b67eee3b19c479cc5b41f10230bf9c1556be6eaaf095c6e997
Successfully built mahotas


In [0]:
#Aqui se importan las librerias necesarias para correr el codigo.
#Antes de correr esta celda se deben ejecutar la anterior para instalar las librerias.
import os
import numpy as np
import nibabel as nib
import csv
import mahotas as mh
import progressbar

%tensorflow_version 1.x
import tensorflow as tf
import time
import scipy.stats as st

In [0]:
#Es la ruta del archivo raiz de los datos preprocesados.
pathArchivoDatosPreprocesados = "Datos"

#Estas 2 variables se utilizan para recorrer cada carpeta de los datos.
sub = [0, 0]
run = 1

#Funcion que retorna el nombre del archivo que contiene las imagenes de resonancia magnetica.
#pSub: Indica la persona a la cual se le tomaron las imagenes.
#pRun: Indica el run para una persona. Cada run incluye 180 muestras de imagenes.
def darNombreArchivoImagenes(pSub, pRun):
  return pathArchivoDatosPreprocesados + "/ds001497-download/sub-"+str(pSub[0])+str(pSub[1])+"/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_bold.nii.gz"

#Funcion que retorna el nombre del archivo que contiene las etiquetas correspondientes a las imagenes de resonancia magnetica.
#pSub: Indica la persona a la cual se le tomaron las imagenes.
#pRun: Indica el run para una persona. Cada run incluye 180 muestras de imagenes.
def darNombreArchivoEtiquetas(pSub, pRun):
  return pathArchivoDatosPreprocesados + "/ds001497-download/sub-"+str(pSub[0])+str(pSub[1])+"/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_events.tsv"

#Funcion que obtiene el conjunto de 30 imagenes de 64x64 utilizados para el proyecto.
def obtenerImagenes(pImagenes):
  
  imagenes = np.zeros((64,64,30,30))

  for indice in range(0,15):
    imagenes[:,:,:,2*indice] = pImagenes[:,:,:,2+indice*13]
    imagenes[:,:,:,2*indice+1] = pImagenes[:,:,:,3+indice*13]

  return imagenes

#Funcion que obtiene las etiquetas y las convierte a su correspondiente numero entero de acuerdo con la siguiente regla:
#object -> 0
#place -> 1
#face -> 2
def leerEtiquetas(pArchivoEtiquetas):
  etiquetas = []

  with open(pArchivoEtiquetas) as tsvfile:
    reader = csv.DictReader(tsvfile, dialect='excel-tab')
    for row in reader:
      
      etiqueta = row['trial_type']

      if(etiqueta == 'object'):

        etiquetas.append(0)
        etiquetas.append(0)
      elif(etiqueta == 'place'):

        etiquetas.append(1)
        etiquetas.append(1)
      elif(etiqueta == 'face'):

        etiquetas.append(2)
        etiquetas.append(2)
      else:
        print("Error en el formato. Existe una clase distinta a object, place o face")

  return etiquetas

#Funcion que genera una matriz de NumPy con los datos leidos de los archivos.
def importarDatos():
  datosPreprocesados = np.zeros((1800,64,64,30))
  etiquetasPreprocesados = np.zeros(1800)

  contador = 0
  p = 0

  sub[0] = 0

  with progressbar.ProgressBar(max_value = 1800) as bar:
    for s1 in range(1,11):#(1,11)

      if(s1 == 10):
        sub[0] = 1
        sub[1] = 0
      else:
        sub[1] = s1

      for run in range(1,7):#(1,7)
        archivoImagenes = darNombreArchivoImagenes(sub, run)
        archivoEtiquetas = darNombreArchivoEtiquetas(sub, run)

        imagenes = nib.load(archivoImagenes).get_fdata()
        etiquetas = leerEtiquetas(archivoEtiquetas)

        muestras = obtenerImagenes(imagenes)

        for indice in range(0,30):
            datosPreprocesados[30*contador+indice,:,:,:] = muestras[:,:,:,indice]
            etiquetasPreprocesados[30*contador+indice] = etiquetas[indice]

            p+=1
            bar.update(p)

        contador+=1

  return datosPreprocesados, etiquetasPreprocesados.astype(int)

In [11]:
#En esta linea de codigo se importan los datos y se almacenan en las variables datosPreprocesados (correspondiente a las imagenes) y en etiquetasPreprocesadas (las etiquetas).
datosPreprocesados, etiquetasPreprocesados = importarDatos()

np.save("drive/My Drive/DatosProyecto/raw/Datos_2_3", datosPreprocesados)
np.save("drive/My Drive/DatosProyecto/raw/Etiquetas_2_3", etiquetasPreprocesados)

100% (1800 of 1800) |####################| Elapsed Time: 0:00:36 Time:  0:00:36


In [0]:
escala = 2**12

#Calcula los descriptores de primer orden. Estos se refieren a los que se sacan a partir del histograma, la media, la varianza, etc.
def calcularDescriptoresPrimerOrden(pImagenes):

  mean = np.mean(pImagenes, axis = None)
  var = np.var(pImagenes, axis = None)
  kur = st.kurtosis(pImagenes, axis = None)
  ske = st.skew(pImagenes, axis = None)

  descriptores = np.append([mean, var],[kur, ske])

  return descriptores

#Calcula los descriptores a partir de un grupo de imagenes de fMRI.
def calcularDescriptores(pImagenes):
  #Faltan los descriptores del histograma
  descPrimer = calcularDescriptoresPrimerOrden(pImagenes)
  descSegundo = mh.features.haralick(pImagenes, ignore_zeros=True, preserve_haralick_bug=False, compute_14th_feature=False, return_mean=True, return_mean_ptp=False, use_x_minus_y_variance=False, distance=1)

  descriptores = np.append(descPrimer,descSegundo)

  return descriptores

#Funcion que retorna el texto de la descripcion del archivo de descriptores.
def darTextoIntroduccion(pEscala):
  texto = "Este archivo incluye los descriptores calculados a partir de los datos preprocesados. \nLos datos brutos fueron obtenidos del \
dataset de imágenes de resonancia magnética funcional que se puede encontrar en https://openneuro.org/datasets/ds001497/versions/1.0.1.\n\
A estos datos se les realizó un preprocesamiento utilizando fmriprep (ESTO TOCA VER SI SI SIRVE AL FIN) y finalmente se calcularon los \
descriptores con una escala de grises de " + str(pEscala) + ".\nEstos descriptores son (EL NUMERO QUE SEA) e incluyen:\nEnergía, etc.... (AGREGAR LOS QUE FALTAN Y ORDENARLOS CORRECTAMENTE).\n\
Finalmente, cada muestra incluye a la clase a la cual pertenece. Existen 3 clases y se indican con un número de 0 a 2 que corresponden a:\n\
objeto -> 0\n\
lugar -> 1\n\
rostro -> 2\n\
media;varianza;kurtosis;skewness;....;clase AGREGAR TODAS LAS VARIABLES ACA EN ORDEN SIGUIENDO LA CONVENCIÓN DE SEPARARLOS CON ;"

  return texto

#Funcion que retorna un string con los descriptores y clases separados por ;.
#Esta cadena sera utilizada para generar el archivo de texto con los descriptores calculados y su etiqueta.
def darLinea(pDescriptores, pEtiqueta):
  separador = ";"
  muestra = []
  return separador.join(pDescriptores.astype(str)) + separador + str(pEtiqueta)

#Funcion que genera el archivo de texto con los datos de los descriptores y su correspondiente etiqueta.
#Este archivo recibe el nombre de datosDescriptores.txt e incluye los descriptores y sus etiquetas separadas por ;
#y una descripcion del archivo.
def crearArchivoDescriptores():

  print("Generando archivo de descriptores...")
  archivo = open("datosDescriptores.txt", "w+")

  intro = darTextoIntroduccion(escala)
  archivo.write(intro)
  archivo.write("\n")

  start = time.time()

  #Esto deberia iterar hasta 1800, pero por ahora menos.
  for i in progressbar.progressbar(range(0,1800)):
    imagenes = datosPreprocesados[:,:,:,i]
    imagenes = escala*(imagenes/imagenes.max())
    imagenes = imagenes.astype(int)

    descriptores = calcularDescriptores(imagenes)
    linea = darLinea(descriptores, etiquetasPreprocesados[i])
    archivo.write(linea)
    archivo.write("\n")
    
  print("Archivo generado exitosamente.")
  print("Tiempo: {} segundos".format(time.time()-start))

  archivo.close()

In [0]:
crearArchivoDescriptores()